In [1]:
import os 
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
import gradio as gr 

ModuleNotFoundError: No module named 'bs4'

In [2]:
# Load environment variables from .env
load_dotenv()

# Define your API keys and expected prefixes
api_keys = {
    "OPEN_AI_API_KEY": "sk-proj-",
    "ANTHROPIC_API_KEY": "sk-ant-",
    "GOOGLE_GEMINI_API_KEY": "gems-"
}

def validate_key(name, prefix):
    key = os.getenv(name)
    if key and key.startswith(prefix):
        print(f"✅ {name} is valid.")
    elif key:
        print(f"⚠️ {name} is present but may be malformed.")
    else:
        print(f"❌ {name} is missing from environment.")
    return key

# Validate and assign keys
openai_key = validate_key("OPEN_AI_API_KEY", api_keys["OPEN_AI_API_KEY"])
anthropic_key = validate_key("ANTHROPIC_API_KEY", api_keys["ANTHROPIC_API_KEY"])
gemini_key = validate_key("GOOGLE_GEMINI_API_KEY", api_keys["GOOGLE_GEMINI_API_KEY"])

# Assign keys to libraries or environment if needed
if openai_key:
    import openai
    openai.api_key = openai_key

if anthropic_key:
    from anthropic import Anthropic
    print("Anthropic SDK loaded successfully")
    # client = Anthropic.Client(api_key=anthropic_key)
    client = Anthropic(api_key=anthropic_key)


if gemini_key:
    import google.generativeai as genai
    genai.configure(api_key=gemini_key)

    os.environ["GOOGLE_GEMINI_API_KEY"] = gemini_key

NameError: name 'load_dotenv' is not defined

In [3]:
system_message = "You are helpful assistant"

In [4]:
def message_gpt(prompt):
    messages = [{"role":"system", "content":system_message},{"role":"user", "content":prompt}]
    completion = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return completion.choices[0].message.content

In [5]:
message_gpt("what is today's date?")

NameError: name 'openai' is not defined

In [6]:
def shout(text):
    print(f"shout has been call with input {text}")
    return text.upper()

In [7]:
shout("Hello World!")

shout has been call with input Hello World!


'HELLO WORLD!'

In [8]:
gr.Interface(fn=shout, inputs="textbox",allow_flagging="never", outputs="textbox").launch(share=True)

NameError: name 'gr' is not defined

In [28]:
view = gr.Interface(
fn=shout,
inputs=[gr.Textbox(label="Your message:", lines=6)],
outputs=[gr.Textbox(label="Response:", lines=8)],
allow_flagging="never"
)
view.launch()

c:\Users\mathe\OneDrive\Desktop\LLM_Course\Large_Language_Model_course_Material\Week2\.venv\lib\site-packages\gradio\interface.py:414: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.


shout has been call with input hello yet again


In [9]:
view = gr.Interface(
fn=shout,
inputs=[gr.Textbox(label="Your message:", lines=6)],
outputs=[gr.Textbox(label="Response:", lines=8)],
allow_flagging="never"
)
view.launch()

NameError: name 'gr' is not defined

In [32]:
system_message = "You are a helpful assistant that respond in markdown"

view = gr.Interface(
    fn=message_gpt,
    inputs=[gr.Textbox(label="Your Message:", lines=6)],
    outputs=[gr.Textbox(label="Your Response:", lines=6)]
)   
view.launch()

* Running on local URL:  http://127.0.0.1:7868
* To create a public link, set `share=True` in `launch()`.


In [10]:
system_message = "You are a helpful assistant that respond in markdown"

view = gr.Interface(
    fn=message_gpt,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Your Response:")],
    allow_flagging="never"
)
view.launch()

NameError: name 'gr' is not defined

In [11]:
def stream_gpt(prompt):
    messages = [
        {"role":"system", "content":system_message},
        {"role":"user", "content":prompt}
    ]
    stream = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result


In [42]:
view = gr.Interface(
    fn=stream_gpt,
    inputs=[gr.Textbox(label="Your message")],
    outputs=[gr.Markdown(label="Response")],
    allow_flagging="never"
)
view.launch()

c:\Users\mathe\OneDrive\Desktop\LLM_Course\Large_Language_Model_course_Material\Week2\.venv\lib\site-packages\gradio\interface.py:414: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7873
* To create a public link, set `share=True` in `launch()`.


In [12]:
def stream_claude(prompt):
    result = client.messages.stream(
        model="claude-3-5-haiku-latest",
        max_tokens=1000,
        temperature=0.7,
        system=system_message,
        messages=[
            {"role": "user", "content": prompt}
        ])
    response = ""
    with result as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response
    

In [44]:
view = gr.Interface(
    fn=stream_claude,
    inputs=[gr.Textbox(label="Your Message")],
    outputs=[gr.Markdown(label="Response")]
)
view.launch()

* Running on local URL:  http://127.0.0.1:7874
* To create a public link, set `share=True` in `launch()`.


In [13]:
def stream_model(prompt,model):
    if model == "GPT":
        result = stream_gpt(prompt)
    elif model == "Claude":
        result = stream_claude(prompt)
    else:
        raise ValueError("Unknown model")
    for chunk in result:
        yield chunk

In [58]:
view = gr.Interface(
    fn=stream_model,
    inputs=[gr.Textbox(label="Your Message:"),gr.Dropdown(["GPT", "Claude"], label="Select Model")],
    outputs=[gr.Markdown(label="Response:")],
    allow_flagging="never"
    )
view.launch()

c:\Users\mathe\OneDrive\Desktop\LLM_Course\Large_Language_Model_course_Material\Week2\.venv\lib\site-packages\gradio\interface.py:414: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7880
* To create a public link, set `share=True` in `launch()`.


In [ ]:
class Website:
    url = str,
    title =str,
    text =str
    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = requests.content
        soup = BeautifulSoup(self.body, "html.parser")
        self.title = soup.title.string if soup.title else "No Title Found"
        for irrelevant in soup.body{["script","style", "img", "input"]}:
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

    def get_content(self):
        return f"Webpage Title:\n{self.title}\nWebpage Content:\n{self.text}\n\n"
        